In [75]:
# !pip install -q kaggle
# import json

# data = {"username":"tunkedsaro","key":"ff3d00dc680ff94caf6f8d4d9bcd3168"}
# json_data = json.dumps(data)

# # Write the JSON string to a file
# file_path = "/content/kaggle.json"
# with open(file_path, 'w') as json_file:
#     json_file.write(json_data)

# !mkdir ~/.kaggle
# !cp /content/kaggle.json ~/.kaggle/ #copy api key ---- depend on your directory -- my directory is .../colab/..
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets list
# !kaggle competitions download -c hackathon-online-data-science-mini-epidemiology
# !unzip -q /content/hackathon-online-data-science-mini-epidemiology.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
User cancelled operation
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/kaggle/cli.py", line 79, in main
    if out is not None:
KeyboardInterrupt
^C
hackathon-online-data-science-mini-epidemiology.zip: Skipping, found more recently modified local copy (use --force to force download)
replace Test_set.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [76]:
# !pip -q install autogluon.tabular

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 15, in <module>
    from pip._vendor.packaging.requirements import Requirement
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/packaging/requirements.py", line 8, in <module>
    from ._parser import parse_requirement as _parse_requirement
  File "/usr/local/lib/python3.10/dist-package

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
import pandas as pd

In [26]:
train_df = pd.read_csv("/content/Train_set.csv")
test_df = pd.read_csv("/content/Test_set.csv")
submit_df = pd.read_csv("/content/sample_submission.csv")

In [27]:
# train_df = train_df[50:60]
train_df

,_id,Disease,Year,PROVINCE,Week_no,Count
0,1,Hand Foot Mouth,2013,Bangkok,1,62.0
1,2,Hand Foot Mouth,2013,Bangkok,2,95.0
2,3,Hand Foot Mouth,2013,Bangkok,3,88.0
3,4,Hand Foot Mouth,2013,Bangkok,4,86.0
4,5,Hand Foot Mouth,2013,Bangkok,5,132.0
...,...,...,...,...,...,...
32181,32182,Hand Foot Mouth,2020,Narathiwat,48,2.0
32182,32183,Hand Foot Mouth,2020,Narathiwat,49,1.0
32183,32184,Hand Foot Mouth,2020,Narathiwat,50,2.0
32184,32185,Hand Foot Mouth,2020,Narathiwat,51,2.0


In [28]:
# test_df = test_df[0:10]
test_df

,_id,Disease,Year,PROVINCE,Week_no
0,32187,Hand Foot Mouth,2021,Bangkok,1
1,32188,Hand Foot Mouth,2021,Bangkok,2
2,32189,Hand Foot Mouth,2021,Bangkok,3
3,32190,Hand Foot Mouth,2021,Bangkok,4
4,32191,Hand Foot Mouth,2021,Bangkok,5
...,...,...,...,...,...
11198,43385,Hand Foot Mouth,2023,Narathiwat,37
11199,43386,Hand Foot Mouth,2023,Narathiwat,38
11200,43387,Hand Foot Mouth,2023,Narathiwat,39
11201,43388,Hand Foot Mouth,2023,Narathiwat,40


In [29]:
combine_df = pd.concat([train_df, test_df])
combine_df

,_id,Disease,Year,PROVINCE,Week_no,Count
0,1,Hand Foot Mouth,2013,Bangkok,1,62.0
1,2,Hand Foot Mouth,2013,Bangkok,2,95.0
2,3,Hand Foot Mouth,2013,Bangkok,3,88.0
3,4,Hand Foot Mouth,2013,Bangkok,4,86.0
4,5,Hand Foot Mouth,2013,Bangkok,5,132.0
...,...,...,...,...,...,...
11198,43385,Hand Foot Mouth,2023,Narathiwat,37,NaN
11199,43386,Hand Foot Mouth,2023,Narathiwat,38,NaN
11200,43387,Hand Foot Mouth,2023,Narathiwat,39,NaN
11201,43388,Hand Foot Mouth,2023,Narathiwat,40,NaN


In [30]:
combine_df = combine_df.drop(columns=['Disease'])
combine_df

,_id,Year,PROVINCE,Week_no,Count
0,1,2013,Bangkok,1,62.0
1,2,2013,Bangkok,2,95.0
2,3,2013,Bangkok,3,88.0
3,4,2013,Bangkok,4,86.0
4,5,2013,Bangkok,5,132.0
...,...,...,...,...,...
11198,43385,2023,Narathiwat,37,NaN
11199,43386,2023,Narathiwat,38,NaN
11200,43387,2023,Narathiwat,39,NaN
11201,43388,2023,Narathiwat,40,NaN


In [31]:
combine_df = pd.get_dummies(combine_df, columns=['PROVINCE'])
combine_df

,_id,Year,Week_no,Count,PROVINCE_Amnat Charoen,PROVINCE_Ang Thong,PROVINCE_Bangkok,PROVINCE_Bueng Kan,PROVINCE_Buriram,PROVINCE_Chachoengsao,...,PROVINCE_Surin,PROVINCE_Tak,PROVINCE_Trang,PROVINCE_Trat,PROVINCE_Ubon Ratchathani,PROVINCE_Udon Thani,PROVINCE_Uthai Thani,PROVINCE_Uttaradit,PROVINCE_Yala,PROVINCE_Yasothon
0,1,2013,1,62.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2,2013,2,95.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3,2013,3,88.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,4,2013,4,86.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,5,2013,5,132.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11198,43385,2023,37,NaN,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11199,43386,2023,38,NaN,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11200,43387,2023,39,NaN,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11201,43388,2023,40,NaN,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [32]:
train_df = combine_df[combine_df['Count'].notna()]
test_df = combine_df[combine_df['Count'].isna()]

<hr>

In [33]:
train_df.columns

Index(['_id', 'Year', 'Week_no', 'Count', 'PROVINCE_Amnat Charoen',
       'PROVINCE_Ang Thong', 'PROVINCE_Bangkok', 'PROVINCE_Bueng Kan',
       'PROVINCE_Buriram', 'PROVINCE_Chachoengsao', 'PROVINCE_Chainat',
       'PROVINCE_Chaiyaphum', 'PROVINCE_Chanthaburi', 'PROVINCE_Chiang Mai',
       'PROVINCE_Chiang Rai', 'PROVINCE_Chonburi', 'PROVINCE_Chumphon',
       'PROVINCE_Kalasin', 'PROVINCE_Kamphaeng Phet', 'PROVINCE_Kanchanaburi',
       'PROVINCE_Khon Kaen', 'PROVINCE_Krabi', 'PROVINCE_Lampang',
       'PROVINCE_Lamphun', 'PROVINCE_Loei', 'PROVINCE_Lopburi',
       'PROVINCE_Mae Hong Son', 'PROVINCE_Maha Sarakham', 'PROVINCE_Mukdahan',
       'PROVINCE_Nakhon Nayok', 'PROVINCE_Nakhon Pathom',
       'PROVINCE_Nakhon Phanom', 'PROVINCE_Nakhon Ratchasima',
       'PROVINCE_Nakhon Sawan', 'PROVINCE_Nakhon Si Thammarat', 'PROVINCE_Nan',
       'PROVINCE_Narathiwat', 'PROVINCE_Nong Bua Lamphu', 'PROVINCE_Nong Khai',
       'PROVINCE_Nonthaburi', 'PROVINCE_Pathum Thani', 'PROVINCE_Patta

In [34]:
my_list = list(train_df.columns)
my_list.remove('_id')
my_list.remove('Count')
print(my_list)

['Year', 'Week_no', 'PROVINCE_Amnat Charoen', 'PROVINCE_Ang Thong', 'PROVINCE_Bangkok', 'PROVINCE_Bueng Kan', 'PROVINCE_Buriram', 'PROVINCE_Chachoengsao', 'PROVINCE_Chainat', 'PROVINCE_Chaiyaphum', 'PROVINCE_Chanthaburi', 'PROVINCE_Chiang Mai', 'PROVINCE_Chiang Rai', 'PROVINCE_Chonburi', 'PROVINCE_Chumphon', 'PROVINCE_Kalasin', 'PROVINCE_Kamphaeng Phet', 'PROVINCE_Kanchanaburi', 'PROVINCE_Khon Kaen', 'PROVINCE_Krabi', 'PROVINCE_Lampang', 'PROVINCE_Lamphun', 'PROVINCE_Loei', 'PROVINCE_Lopburi', 'PROVINCE_Mae Hong Son', 'PROVINCE_Maha Sarakham', 'PROVINCE_Mukdahan', 'PROVINCE_Nakhon Nayok', 'PROVINCE_Nakhon Pathom', 'PROVINCE_Nakhon Phanom', 'PROVINCE_Nakhon Ratchasima', 'PROVINCE_Nakhon Sawan', 'PROVINCE_Nakhon Si Thammarat', 'PROVINCE_Nan', 'PROVINCE_Narathiwat', 'PROVINCE_Nong Bua Lamphu', 'PROVINCE_Nong Khai', 'PROVINCE_Nonthaburi', 'PROVINCE_Pathum Thani', 'PROVINCE_Pattani', 'PROVINCE_Phang Nga', 'PROVINCE_Phatthalung', 'PROVINCE_Phayao', 'PROVINCE_Phetchabun', 'PROVINCE_Phetchabur

In [35]:
# Split the dataset
X = train_df[my_list]
# X = train_df[list(train_df.columns)]
y = train_df['Count']

In [36]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
# Train a RandomForest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [38]:
# Predict and evaluate
y_pred = model.predict(X_val)
rms = np.sqrt(mean_squared_error(y_val, y_pred))
rms

9.268046959271778

<hr>

In [39]:
submit_df

,_id,Pred
0,32187,32.0
1,32188,13.0
2,32189,17.0
3,32190,NaN
4,32191,NaN
...,...,...
11198,43385,NaN
11199,43386,NaN
11200,43387,NaN
11201,43388,NaN


In [40]:
X_train

,Year,Week_no,PROVINCE_Amnat Charoen,PROVINCE_Ang Thong,PROVINCE_Bangkok,PROVINCE_Bueng Kan,PROVINCE_Buriram,PROVINCE_Chachoengsao,PROVINCE_Chainat,PROVINCE_Chaiyaphum,...,PROVINCE_Surin,PROVINCE_Tak,PROVINCE_Trang,PROVINCE_Trat,PROVINCE_Ubon Ratchathani,PROVINCE_Udon Thani,PROVINCE_Uthai Thani,PROVINCE_Uttaradit,PROVINCE_Yala,PROVINCE_Yasothon
11194,2015,42,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
13235,2016,3,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
32032,2020,3,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
21237,2018,24,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
1580,2013,44,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29802,2020,9,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5390,2014,10,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
860,2013,13,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15795,2016,15,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [41]:
model.predict(test_df[test_df['_id']==32191].drop(columns=['_id','Count']))

array([61.24])

In [42]:
df = pd.read_csv("/content/sample_submission.csv")
df

,_id,Pred
0,32187,32.0
1,32188,13.0
2,32189,17.0
3,32190,NaN
4,32191,NaN
...,...,...
11198,43385,NaN
11199,43386,NaN
11200,43387,NaN
11201,43388,NaN


In [43]:
test_df

,_id,Year,Week_no,Count,PROVINCE_Amnat Charoen,PROVINCE_Ang Thong,PROVINCE_Bangkok,PROVINCE_Bueng Kan,PROVINCE_Buriram,PROVINCE_Chachoengsao,...,PROVINCE_Surin,PROVINCE_Tak,PROVINCE_Trang,PROVINCE_Trat,PROVINCE_Ubon Ratchathani,PROVINCE_Udon Thani,PROVINCE_Uthai Thani,PROVINCE_Uttaradit,PROVINCE_Yala,PROVINCE_Yasothon
0,32187,2021,1,NaN,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,32188,2021,2,NaN,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,32189,2021,3,NaN,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,32190,2021,4,NaN,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,32191,2021,5,NaN,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11198,43385,2023,37,NaN,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11199,43386,2023,38,NaN,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11200,43387,2023,39,NaN,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11201,43388,2023,40,NaN,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [44]:
df

,_id,Pred
0,32187,32.0
1,32188,13.0
2,32189,17.0
3,32190,NaN
4,32191,NaN
...,...,...
11198,43385,NaN
11199,43386,NaN
11200,43387,NaN
11201,43388,NaN


In [45]:
df = pd.read_csv("/content/sample_submission.csv")
for idx,row in df[:].iterrows():
    data = test_df[test_df['_id']==row['_id']].drop(columns=['_id','Count'])
    result = model.predict(data)
    df.at[idx,'Pred'] = result

In [46]:
df

,_id,Pred
0,32187,64.75
1,32188,66.58
2,32189,75.10
3,32190,68.89
4,32191,61.24
...,...,...
11198,43385,78.95
11199,43386,71.46
11200,43387,41.26
11201,43388,25.06


In [47]:
version = '0.0.1'
formatted_time = time.strftime("%H.%M", time.localtime())
formatted_time = str(int(formatted_time.split(".")[0])+7)+formatted_time.split(".")[1]
formatted_time
csv_name = f'v.{version}.{formatted_time}.csv'
print(csv_name)
df.to_csv(csv_name,index=False)

v.0.0.1.2330.csv
